In [1]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
import glob
import os
from tqdm import tqdm

2023-08-15 08:40:32.362785: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 08:40:32.793689: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 08:40:32.795402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 08:40:35.576748: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
def pred_by_teacheble_machine(model, class_names, image_path: str) -> float:
    """Teacheble Machineで作成したモデルで予測
    """
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    np.set_printoptions(suppress=True)
    image = Image.open(image_path).convert("RGB")

    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    
    image_array = np.asarray(image)
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    
    # Load the image into the array
    data[0] = normalized_image_array
    
    # Predicts the model
    prediction = model.predict(data)

    return prediction[0][1]

## Teachable Machineでのmain文

In [7]:
TEST_IMAGE_DIR_PATH = "../Dataset/test/*"
MODEL_PATH = "../Models/TeachableMachine/500epoch_normal/keras_model.h5"
LABEL_PATH = "../Models/TeachableMachine/500epoch_normal/labels.txt"

# Load the model
model = load_model(MODEL_PATH, compile=False)

# Load the labels
class_names = open(LABEL_PATH, "r").readlines()

file_names = []
preds = []

for image_file_path in tqdm(glob.glob(TEST_IMAGE_DIR_PATH)):
    file_name = os.path.basename(image_file_path)
    pred = pred_by_teacheble_machine(model, class_names, image_file_path)
    
    file_names.append(file_name)
    preds.append(pred)

d = {'0': file_names, '1': preds}
dst_df = pd.DataFrame(data=d)
export_path = "../output/20230815TM500epoch_liner.csv"
dst_df.to_csv(export_path, index=False, header=False)

1/1 [==============================] - 0s 33ms/step
